In [2]:
from bitmex import bitmex
import json
from datetime import datetime as dt
import pandas as pd
import warnings
from swagger_spec_validator.common import SwaggerValidationWarning
import numpy as np
import time
from tqdm.notebook import tqdm, trange

if __name__ == "__main__":
    print("Hello_World")
    REMOTE = False
    warnings.simplefilter("ignore", SwaggerValidationWarning)



Hello_World


In [3]:
client = bitmex(test=True)
# total_size = 1_000_000
start_index = 20_000_000
total_size = 2_000_000
split_size = 50_000
pull_size = 1_000
pull_count = int(split_size/pull_size)
split_count = int(total_size/split_size)


In [4]:
split_count

40

In [16]:
pull_count

50

In [5]:
def pull(start=0, size = 100):
    return client.Trade.Trade_get(symbol="XBT", count=size, start=start)

In [15]:
file_name = "Bitmex_Hist.csv"
remaining = 12
for j in trange(0, split_count):
    storage_df = pd.json_normalize(pull(start=0, size = pull_size).result()[0])
    for i in trange(1, pull_count):
        starting = (i*pull_size) + start_index
        while True:
            if int(remaining) < 10:
                print(remaining)
                time.sleep(15)
                if int(remaining) < 2:
                    time.sleep(120)
            try:
                pulled = pull(start=starting, size = pull_size) 
                df_temp =  pd.json_normalize(pulled.result()[0])
                remaining = pulled.response().metadata.headers['X-RateLimit-Remaining']
                break
            except:
                print(remaining)
                print("In cool-down at pull {} of split {}".format(i,j))
                time.sleep(600)
                
        df_temp.index = range(starting,starting+pull_size)
        storage_df = storage_df.append(df_temp)
        time.sleep(1)
    if(j == 0):
        storage_df.to_csv(file_name)
        print("Stored split: {}  - Remaining: {}".format(j, remaining))
    else:
        storage_df.to_csv(file_name, mode='a', header=False)
        print("Stored split: {}  - Remaining: {}".format(j, remaining))

9
9
9
8
9
9

Stored split: 0  - Remaining: 11


9
9
8
9
9
9
8
9
9

Stored split: 1  - Remaining: 12


9
9
9
9
8
9
9
9
9

Stored split: 2  - Remaining: 12


9
9
9
9
9
9
9
9


KeyboardInterrupt: 